<a href="https://colab.research.google.com/github/taekyungss/Hands_on_ML_Study/blob/main/Chapter2_housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **chapter2. 머신러닝 프로젝트 처음부터 끝까지**

### 데이터 다운로드

In [ ]:
import os
import tarfile
import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/rickiepark/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url = HOUSING_URL, housing_path = HOUSING_PATH):
  os.makedirs(housing_path,exist_ok =True)
  tgz_path = os.path.join(housing_path, "housing.tgz")
  urllib.request.urlretrieve(housing_url, tgz_path)
  housing_tgz = tarfile.open(tgz_path)
  housing_tgz.extractall(path = housing_path)
  housing_tgz.close()

In [ ]:
fetch_housing_data()

In [ ]:
# 판다스 사용해 데이터 읽어오기 - 데이터 프레임 객체 반환

In [ ]:
import pandas as pd

def load_housing_data(housing_path = HOUSING_PATH):
  csv_path = os.path.join(housing_path,"housing.csv")
  return pd.read_csv(csv_path)

### 데이터 구조 흝어보기

In [ ]:
# housing 변수에 불러온 데이터 저장, 5행 불러오기.
housing = load_housing_data()
housing.head()

In [ ]:
# feature는 longitude, longitude housing_median_age total_rooms total_bedrooms~

In [ ]:
# 데이터에 대한 간단한 설명

housing.info()

In [ ]:
# ocean_proximity 의 경우, Dtype이 object, 아마 범주형 추정
# 어떤 카테고리가 있고 얼마나 많은 구역이 있는지 확인

housing["ocean_proximity"].value_counts()


In [ ]:
# descrobe method : 숫자형 특성의 요약 정보 나타냄

housing.describe()

In [ ]:
# 숫자형 데이터에 대해 히스토그램으로 시각화해보기

%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=50,figsize=(10,8))
plt.show()

In [ ]:
# housing_median_age , median_house_value feature들을 봤을때, 젤 오른쪽 값이 급격하게 높아지면서 히스토그램이 끝남.
# 이는 마지막 값으로 한정 (한계값을 넘어가지 않도록 미리 최댓값을 정해서 설정해놓은것을 알 수 있음. ) 이 부분이 문제가 되는 지 안되는지 검토할 필요가 있음.
# 만약 정확한 예측값이 필요한 경우, a. 한곗값 밖의 구역에 대한 정확한 레이블을 구한다. b. 훈련 세트에서 이런 구역을 제거한다. (테스트세트에서도 제거)

### 테스트 세트 만들기

순수한 무작위 방식

In [ ]:
import numpy as np
np.random.seed(42)

In [ ]:

def split_train_test(data, test_ratio):
  shuffled_indices = np.random.permutation(len(data))
  test_set_size = int(len(data)*test_ratio)
  test_indices = shuffled_indices[:test_set_size]
  train_indices = shuffled_indices[test_set_size:]
  return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
train_set, test_set = split_train_test(housing, 0.2)

print(len(train_set))
print(len(test_set))

In [ ]:
# 프로그램을 다시 실행하면 다른 테스트 세트 생성 , 
# --> 알고리즘이 전체 데이터셋을 볼 수도.. 따라서 기존 훈련과 test를 완벽히 분리해야함.
# 이를 구현한 코드
import numpy as np

from zlib import crc32

def test_set_check(identifier, test_ratio):
  return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32

def split_train_test_by_id(data, test_ratio, id_column):
  ids = data[id_column]
  in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
  return data.loc[~in_test_set], data.loc[in_test_set]
  


In [ ]:
import hashlib

def test_set_check(identifier, test_ratio, hash=hashlib.md5):
    return hash(np.int64(identifier)).digest()[-1] < 256 * test_ratio

In [ ]:
housing_with_id = housing.reset_index()

train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index")

In [ ]:
housing_with_id["id"] = housing["longitude"] * 1000 + housing["latitude"]
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "id")

In [ ]:
# 사이킷런을 통해 데이터셋을 train, test로 분리

from sklearn.model_selection import train_test_split

train_set, test_set= train_test_split(housing, test_size=0.2, random_state=42)

계층적 샘플링 방식

특정 방식을 유지하면서 샘플 추출

In [ ]:
# 전문가들이 중간 소득이 중간 주택 가격을 예측하는데 익어서 매우 중요하다고 한다. 
# 이 경우, 중간 소득의 비율을 유지하면서 / 즉, 테스트 세트가 전체 데이터셋에 있는 여러 소득 카테고리를 잘 대표해야한다.

In [ ]:
housing["median_income"].hist()

In [ ]:
housing["income_cat"] = pd.cut(housing["median_income"], 
                               bins=[0.,1.5,3.0,4.5,6., np.inf], 
                               labels=[1,2,3,4,5])

In [ ]:
housing["income_cat"].value_counts()

In [ ]:
housing["income_cat"].hist()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
  strat_train_set = housing.loc[train_index]
  strat_test_set = housing.loc[test_index]

In [ ]:
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

In [ ]:
housing["income_cat"].value_counts() / len(housing)

In [ ]:
housing["income_cat"]

In [ ]:
# income_cat특성 삭제, 데이터 원래 상태로 복구

for set_ in (strat_train_set, strat_test_set):
  set_.drop("income_cat", axis=1, inplace=True)

### 데이터 이해를 위한 탐색과 시각화

In [ ]:
# 훈련세트 손상 시키지 않기 위해 복사본 만들어 사용

housing = strat_train_set.copy()
# print(housing)

# 특성 및 레이블 분리

# housing = strat_train_set.drop("median_house_value", axis = 1)
# housing_labels = strat_train_set["median_house_value"].copy()

In [ ]:
# 지리적 데이터 시각화 ( 지리정보 ) --> 산점도

housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)

# alpha 옵션 -> 데이터 포인트가 밀집된 영역을 잘 보여줌.

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
             s=housing["population"]/100, label="population", figsize=(10,7),
             c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True, 
             sharex=False
             )
plt.legend()

In [ ]:
# 위 시각화 그림을 통해 주액 가격은 지역과 인구 밀도에 관련이 크다는 것을 확인 가능.

### 상관관계 조사

In [ ]:
# 표준 상관계수(피어슨의 r) corr() method 사용

corr_matrix = housing.corr()

In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
from ast import Attribute
# scatter_matrix 함수 사용해서 산점도 그려서 상관관계 확인

from pandas.plotting import scatter_matrix
attributes = ["median_house_value","median_income", "total_rooms","housing_median_age"]
scatter_matrix(housing[attributes], figsize=(8,5))

In [ ]:
# median_house_value을 예측하는데 중간소득이 가장 유용 해당 상관관계 산점도

housing.plot(kind="scatter", x="median_income", y="median_house_value",
             alpha=0.1)


In [ ]:
# 위 그래프에서 직선에 가까운 수평선이 잘 보임. 이런 이상한 형태 학습하지 않도록 해당 구역 제거할 필요 있음.

### 특성 조합으로 실험

In [ ]:
housing.head()

In [ ]:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"] = housing["population"]/housing["households"]

In [ ]:
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
housing.plot(kind="scatter", x="rooms_per_household", y="median_house_value",
             alpha = 0.2)
plt.show()

### 데이터 정제

In [ ]:
# 알고리즘을 위한 데이터 준비

housing = strat_train_set.drop("median_house_value", axis=1) # 훈련 세트를 위해 레이블 삭제
housing_labels = strat_train_set["median_house_value"].copy()

In [ ]:
housing.info() # null값 존재 이제 처리

In [ ]:
# 처리방법 3가지 1. 해당 구역 제거 2. 전체 특성 삭제 3. 어떤 값으로 채우기(0, 평균, 중간값 등.)

# housing.dropna(subset=["total_bedrooms"]) # 옵션1
# housing.drop("total_bedrooms", axis=1)  # 옵션2
# median = housing["total_bedrooms"].median()
# housing["total_bedrooms"].fillna(median,inplace=True)  # 옵션3

In [ ]:
# 사이킷런의 SimpleImputer을 사용해 누락된 값 손쉽게 다루기 _ 중앙값으로 결측지 채우기

from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy ="median")
housing_num = housing.drop("ocean_proximity", axis=1)
imputer.fit(housing_num)

In [ ]:
# 어떤 값을 저장했는지 확인
imputer.statistics_

In [ ]:
housing_num.median().values

In [ ]:
X = imputer.transform(housing_num)

In [ ]:
housing_tr = pd.DataFrame(X, columns = housing_num.columns,
                          index=housing_num.index)

In [ ]:
# housing_tr.loc[sample_incomplete_rows.index.values]

imputer.strategy
housing_tr.head()

### 텍스트와 범주형 특성 다루기

In [ ]:
housing_cat = housing[["ocean_proximity"]]
housing_cat.head(10)

In [ ]:
# 해당 카테고리를 텍스트에서 숫자로 변환

from sklearn.preprocessing import OrdinalEncoder

In [ ]:
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

In [ ]:
# 카테고리 목록 얻기

ordinal_encoder.categories_

In [ ]:
# One_hot encoding

from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

In [ ]:
# one-hot encoding 은 사이파이 희소행렬 sparse matrix 수천 개의 카테고리가 있는 범주형 특성일 경우 효율정

housing_cat_1hot.toarray()

In [ ]:
# 카테고리 얻기

cat_encoder.categories_

### 나만의 변환기 만들기

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

# 열 인덱스
rooms_ix, bedrooms_ix, population_ix, households_ix = 3,4,5,6

class CombinedAttributeAdder(BaseEstimator,TransformerMixin ):
  def __init__(self, add_bedrooms_per_room=True):
    self.add_bedrooms_per_room = add_bedrooms_per_room
  def fit(self, X, y=None):
    return self
  def transform(self,X):
    rooms_per_household = X[:, rooms_ix ]
    population_per_household = X[:,population_ix]/ X[:,households_ix]
    if self.add_bedrooms_per_room:
      bedrooms_per_room = X[:,bedrooms_ix] / X[:,rooms_ix]
      return np.c_[X, rooms_per_household,population_per_household ,bedrooms_per_room ]

    else:
      return np.c_[X, rooms_per_household,population_per_household ]

attr_adder= CombinedAttributeAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

(1) 두 배열을 왼쪽에서 오른쪽으로 붙이기 

   : np.r_[a, b]

   : np.hstack([a, b])
   : np.concatenate((a, b), axis = 0)



(2) 두 배열을 위에서 아래로 붙이기

   : np.r_[[a], [b]]

   : np.vstack([a, b])

   : np.concatenate((c, d), axis = 1) # for 2D ~ array



(3) 두 개의 1차원 배열을 칼럼으로 세로로 붙여서 2차원 배열 만들기

    (Stack 1-D arrays as columns into a 2-D array)

   : np.c_[a, b]

   : np.column_stack([a, b])

   : np.concatenate((c.T, d.T), axis = 1) # for 2D~ array

### 특성 스케일링

모든 특성의 범위를 같도록 만드렁주는 방법 1. min-max 스케일링,2. 표준화

1. min-max 스케일링


 0-1사이에 들도록 값을 이동하고 스케일을 조정한것

데이터에서 최솟값을 뺀 후, 최댓값과 최솟값의 차이로 나누기.

2. 표준화


먼저 평균을 뺀후, (표준을 하면 평균이 0 - 우리가 아는 정규분포처럼 만드는게 

표준화? ) 이는 범위의 상한과 하한이 없어 특정 알고리즘에서는 문제가 될 수 있다.


> 


**하지만.. ! 만약 이상치가 존재할 경우에는 표준화가 영향을 덜 받고**

**min-max 스케일링은 이상치에 영향을 받기 때문에.. 이상치를 고려해서 이 둘의 스케일링을 선택하도록 해야한다. **


### 변환 파이프라인

 변환 단계가 많을 경우, 정확한 순서대로 실행되어야함...!

 사이킷런의 연속된 변환을 순서대로 처리할 수 있도록 하는  pipeline 클래스 이용

**숫자 특성을 처리하는 파이프라인**


In [ ]:
# 처음부터 변환을 시도할때 이러한 파이프라인을 사용해서 코딩을 짜면 보다 효율적이고 정확한 순서대로 변환을 처리할 수 있다.

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy = "median")),
    ('attribs_adder', CombinedAttributeAdder()),
    ("std_scaler", StandardScaler())
])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [ ]:
housing_num_tr

In [ ]:
# ColumnTransformer 등장 / for 사이킷런
# 하나의 변환기로 각 열마다 적절한 변환을 적용해 모든 열을 처리할 수 있도록 한다 .

from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
    ('num', num_pipeline,num_attribs ),
    ('cat', OneHotEncoder(), cat_attribs)
])

housing_prepared = full_pipeline.fit_transform(housing)

# num - 밀집 행렬 반환
# cat - 희소 행렬 반환

In [ ]:
housing_prepared

In [ ]:
housing_prepared.shape

### 모델 선택과 훈련

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)


In [ ]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print("예측: ",lin_reg.predict(some_data_prepared))


In [ ]:
print("레이블: ", list(some_labels))

In [ ]:
some_data_prepared

In [ ]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(housing_prepared, housing_labels)

In [ ]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

오차 존재 x -> 과대적합... ㅎㅎ

In [ ]:
# 교차검증을 사용한 평가

from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)

tree_rmse_scores = np.sqrt(-scores)

In [ ]:
def display_scores(scores):
  print("점수: ", scores)
  print("평균: ", scores.mean())
  print("표준편차: ", scores.std())

In [ ]:
display_scores(tree_rmse_scores)

In [ ]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                             scoring="neg_mean_squared_error", cv=10)

lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
forest_reg.fit(housing_prepared, housing_labels)

In [ ]:
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

In [ ]:
from sklearn.model_selection import cross_val_score

forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                             scoring="neg_mean_squared_error", cv=10)

forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

In [ ]:
scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
pd.Series(np.sqrt(-scores)).describe()

In [ ]:
from sklearn.svm import SVR

svm_reg = SVR(kernel="linear")
svm_reg.fit(housing_prepared, housing_labels)
housing_predictions = svm_reg.predict(housing_prepared)
svm_mse = mean_squared_error(housing_labels, housing_predictions)
svm_rmse = np.sqrt(svm_mse)
svm_rmse

### 모델 세부 튜닝

그리드 탐색

simple method: 만족할 만한 하이퍼파라미터 조합을 찾을때까지 수동으로 조정하는거

-> 자동화: 사이킷런의 GridSearchCV사용하기
(탐색하고자 하는 하이퍼파라미터와 시도해볼 값을 지정만 하면 가능)

In [ ]:
# RandomForestRegressor에 대한 최적의 하이퍼파라미터 조합 탐색하기

from sklearn.model_selection import GridSearchCV

param_grid = [
    {"n_estimators": [3,10,30], "max_features": [2,4,6,8]},
    {"bootstrap":[False], "n_estimators":[3,10], "max_features": [2,3,4]},
]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring = "neg_mean_squared_error",
                           return_train_score =True)

grid_search.fit(housing_prepared, housing_labels)

In [ ]:
grid_search.best_params_

In [ ]:
#  최적의 추정기에 직접 접근도 가능

grid_search.best_estimator_

In [ ]:
# 평가점수 확인

cvres= grid_search.cv_results_

for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
  print(np.sqrt(-mean_score), params)

50055.90991344175 {'max_features': 6, 'n_estimators': 30}

이 예시에서는 max_features의 하이퍼파리미터가 6, n_estimators의 하이퍼파라미터가 30일때 최적의 솔루션임을 확인가능


... 뭔가 잘못되었는뎅ㅂ...ㅎ? 